In [2]:
import os
import openai
from dotenv import load_dotenv
from datetime import datetime
from textwrap import dedent
from pydantic import BaseModel
import json



load_dotenv()

#API_KEY = os.getenv('API_KEY')
API_KEY_CGPT = os.getenv('API_KEY_CGPT')

print(f'API ChatGPT: ',{API_KEY_CGPT})

openai.api_key = API_KEY_CGPT

API ChatGPT:  {'sk-proj-FrNMaLZT7tgBftIDPuBOT3BlbkFJFND9eOXtUMeCyWxplFlV'}


In [3]:
def call_gpt_api_with_single_prompt(instructions, prompt, model="gpt-4o-2024-08-06", max_tokens=2500, response_format=None, img_path=None,detail='high'):
    """
    Sends a single message to GPT API with optional image input and retrieves the response.
    
    Parameters:
    - instructions: System instructions to set the context (e.g., "You are an AI assistant that analyzes tables").
    - prompt: User's message or query (e.g., "Please analyze the table in the image and provide a summary").
    - model: The GPT model to be used (default is "gpt-4o-2024-08-06").
    - max_tokens: Maximum number of tokens for the response (default is 2500).
    - response_format: Format of the response (e.g., "Rag_reponse"). Defaults to standard completion if not provided.
    - img_path: Optional path to an image file. If provided, the image will be included in the request.
    
    Returns:
    - The GPT answer object.
    """

    content = []
    dict_images = []
    # Create the messages list to send to GPT
    messages = [
        {"role": "system", "content": instructions}
    ]

    # If an image path is provided, encode and append it as a separate message
    if img_path:
        base64_image = encode_image(img_path)
        prompt_text = {'type':'text','text':dedent(prompt)}
        dic_images = {'type':'image_url','image_url':{'url': f"data:image/png;base64,{base64_image}",'detail':detail}}
        dict_images.append(dic_images)
        content.append(prompt_text)
        content.extend(dict_images)
        chat = {"role": "user", "content":content}

    else:
        chat = {"role": "user", "content":dedent(prompt)}
    
    messages.append(chat)
    
    try:
        if response_format == None:
            # Call GPT API using OpenAI's beta chat completions with parse
            response = openai.beta.chat.completions.parse(
                model=model,
                messages=messages,
                max_tokens=max_tokens)
        else:
            # Call GPT API using OpenAI's beta chat completions with parse
            response = openai.beta.chat.completions.parse(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            response_format=response_format)

        # Extract and return the response content
        answer = response.choices[0].message.content
        return answer

    except Exception as e:
        print(f"Error during GPT API call: {e}")
        return None

def encode_image(image_path):
    """
    Encodes an image to base64 for transmission.
    
    Parameters:
    - image_path: The path to the image to encode.
    
    Returns:
    - Base64 encoded image as a string.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [26]:
class user_query_format(BaseModel):
    prompts : list[str]
    key_terms: list[str]
    date_retrieved : str


def preprocess_user_query(query):

    system_instructions = """You are a specialized system that analyzes user queries and helps extract important information of the query. Do the following 3 tasks: Generate alterantive user prompt, extract key terms and synonyms of user's query and extract date of the query that user is referring to.
    1.	Intent Recognition:
    - Identify if the user is asking about requirements, funding, compliance, or technical details.
    - Focus on specific terms related to technology (e.g., Wärmepumpen, U-Wert).
	2.	Key Terms:
    - Extract key phrases related to the subject (e.g., Wärmepumpen, Förderungen, U-Wert).
	3.	Generate Alternate Queries:
    - Create 3 alternate queries using:
	    1.	Synonyms or similar terms.
	    2.	Different sentence structures.
	    - Ensure they maintain the user’s intent.
    4. Extract key terms mentioned in the prompt, in which the user is interested of knowing about. Give a list of the terms and some synonyms.
    5. Based on the user's prompt, infer the date which is the question talking about. If no date is inferred, respond: "No date found", if yes, say the date in the following format: dd/mm/yyyy. Just answer with the date or the text. When no exact day can inferred, assume last day of the given month. If just the year is mention, say the last day of the year.
    If a date in the future is mentioned, return that no date was found. 
	6.	Output:
    - List of alternative generated prompts.
    - List of key terms and synonyms.
    - Extracted date of the user's prompt."""

    prompt = f"Analyze the user's question: {query}"

    answer = call_gpt_api_with_single_prompt(instructions=system_instructions,
                                             prompt= prompt,
                                             response_format= user_query_format
                                             )

    answer = json.loads(answer)
    prompts = answer['prompts']
    keyterms = answer['key_terms']
    date_retrieved = answer['date_retrieved']

    return answer

In [27]:
query = "Welche Voraussetzungen gelten für Multi-Split-Geräte in Bezug auf die Förderfähigkeit?"
query = "(April 2024) Wie wird die Netzdienlichkeit einer Wärmepumpe nachgewiesen?"

answer = preprocess_user_query(query)

In [28]:
answer_json = json.loads(answer)

answer_json

{'prompts': ['Wie zeigt man die Netzdienlichkeit einer Wärmepumpe?',
  'Was sind die Anforderungen für die Netzdienlichkeit einer Heizungspumpe?',
  'Wie lässt sich die Netztauglichkeit von Wärmepumpen belegen?'],
 'key_terms': ['Netzdienlichkeit', 'Wärmepumpe'],
 'date_retrieved': 'No date found'}

In [ ]:
def update_date_conversation(user_chat,query='Text'):

    system_instructions = dedent('''Based on a given text, infer the date which is the question talking about. If no date is inferred, respond: "No date found", if yes, say the date in the following format: dd/mm/yyyy. Just answer with the date or the text. When no exact day can inferred, assume last day of the given month. If just the year is mention, say the last day of the year.
    If a date in the future is mentioned, return that no date was found. ''')

    query_send = f'''The text is the following:
    {query}'''

    retrieved_date = simple_llm_query(text=dedent(query_send),system_instruction=system_instructions)
    ## Clean date
    retrieved_date = retrieved_date.strip()
    

    if retrieved_date.replace('.','').lower() != 'no date found':
        # print(f'Date was updated with: {retrieved_date}')
        retrieved_date = datetime.strptime(retrieved_date, '%d/%m/%Y')
        user_chat.update_project_date(retrieved_date)
    # else:
    print(retrieved_date)
    
    return user_chat.date_project

In [29]:
test = {'ids': [['34.172', '19.96', '75.17', '34.176', '34.215', '34.36', '34.138', '19.91', '34.170', '34.173', '75.32', '34.136', '34.119', '34.198', '19.159', '34.199', '19.158', '1.83', '1.81', '19.88', '19.141', '19.104', '75.22', '19.76', '34.53'], ['34.176', '34.172', '34.215', '19.96', '34.138', '34.36', '34.119', '19.88', '1.6', '19.159', '34.199', '75.17', '8.23', '19.79', '34.108', '34.170', '19.91', '34.136', '19.158', '1.56', '34.186', '34.173', '75.32', '1.33', '34.53'], ['34.215', '19.158', '19.159', '19.155', '34.41', '34.170', '19.104', '19.208', '34.176', '19.75', '1.41', '34.108', '34.36', '75.9', '19.79', '34.119', '19.105', '34.138', '19.100', '19.96', '19.69', '75.17', '34.212', '34.53', '8.17'], ['34.176', '34.36', '19.158', '1.139', '1.30', '34.172', '19.155', '1.28', '75.9', '1.33', '1.56', '34.119', '1.11', '19.88', '19.159', '75.167', '34.230', '34.56', '34.53', '75.11', '1.6', '8.24', '1.32', '19.171', '34.108']], 'distances': [[306.4400634765625, 324.4844970703125, 333.81231689453125, 341.6695251464844, 342.33734130859375, 347.9888000488281, 348.6868896484375, 361.77899169921875, 362.77880859375, 368.6402282714844, 369.094482421875, 369.2611083984375, 371.5230407714844, 376.7248229980469, 378.7490539550781, 380.2325744628906, 380.68121337890625, 380.72222900390625, 383.75115966796875, 385.03985595703125, 386.53131103515625, 389.07135009765625, 390.14306640625, 394.29266357421875, 395.7266845703125], [300.02850341796875, 311.5758361816406, 323.07696533203125, 325.3227233886719, 328.95074462890625, 331.9451904296875, 332.3088684082031, 341.3415222167969, 341.8349609375, 346.8504638671875, 348.0629577636719, 348.58795166015625, 356.8649597167969, 358.3850402832031, 359.21380615234375, 361.2652893066406, 361.5179138183594, 361.6552429199219, 362.2878723144531, 363.3045349121094, 364.0228576660156, 365.30181884765625, 366.3609619140625, 369.1232604980469, 375.6680603027344], [272.9100036621094, 293.03802490234375, 325.55194091796875, 326.104248046875, 337.01556396484375, 337.91131591796875, 337.97747802734375, 340.1511535644531, 340.78082275390625, 341.85821533203125, 343.13543701171875, 346.6239013671875, 349.10028076171875, 349.24847412109375, 352.0572509765625, 353.5679016113281, 360.23431396484375, 363.0555419921875, 363.34228515625, 363.6197204589844, 366.8296203613281, 366.8879699707031, 368.5907287597656, 369.4739685058594, 370.37298583984375], [267.72259521484375, 273.7691955566406, 283.16741943359375, 288.9434814453125, 291.9920959472656, 293.2082214355469, 294.5688171386719, 299.7608642578125, 300.668701171875, 304.6638488769531, 305.9850769042969, 308.7441101074219, 315.0906677246094, 315.3878479003906, 315.85137939453125, 317.2148742675781, 317.32989501953125, 317.501708984375, 317.775634765625, 319.6430358886719, 319.787353515625, 320.5375671386719, 322.5933837890625, 322.6011657714844, 322.9040222167969]], 'metadatas': None, 'embeddings': None, 'documents': None, 'uris': None, 'data': None}

In [38]:
test['ids'][0]
y = []
y.extend(test['ids'][0])
y.extend(test['ids'][0])
print(len(y))

200


In [40]:
list_t = ["1.28", "19.123", "75.93", "19.130", "19.124", "19.134", "1.139", "19.114", "75.52", "1.130", "19.131", "1.30", "19.156", "19.120", "19.115", "75.9", "34.184", "19.144", "19.188", "19.158", "8.22", "1.125", "19.129", "19.116", "1.11", "1.28", "75.93", "19.130", "19.123", "1.139", "19.114", "75.52", "1.130", "8.22", "34.176", "19.144", "34.82", "19.134", "34.184", "19.124", "19.131", "8.24", "19.188", "34.60", "1.30", "1.125", "34.115", "34.35", "8.27", "1.45", "19.123", "1.28", "75.93", "19.114", "19.124", "19.130", "1.139", "75.52", "19.134", "1.130", "19.120", "19.144", "19.131", "19.188", "8.22", "34.184", "1.125", "1.30", "19.115", "34.176", "19.118", "19.129", "34.82", "19.119", "1.119", "75.93", "1.28", "19.123", "75.52", "1.139", "19.130", "19.114", "19.131", "19.134", "1.130", "1.125", "19.124", "34.184", "19.188", "75.167", "1.30", "8.24", "19.111", "34.115", "8.22", "19.115", "34.176", "1.121", "75.38", "1.119"]

from collections import Counter


# Count occurrences of each element
count = Counter(list_t)

# Filter to keep only items that are duplicates (appear more than once)
duplicates = {item: cnt for item, cnt in count.items() if cnt > 1}

# Number of items that are duplicates
num_duplicate_items = len(duplicates)

# Display results
print(f"Number of items that are duplicates: {num_duplicate_items}")
print("Duplicate items and their counts:")
for item, cnt in duplicates.items():
    print(f"Item: {item}, Count: {cnt}")

# Remove duplicates by converting the list to a set
unique_list = list(set(list_t))

# Count how many unique values there are
num_unique_values = len(unique_list)

# Display the unique list and count
print(f"Unique list: {unique_list}")
print(f"Number of unique values: {num_unique_values}")

Number of items that are duplicates: 25
Duplicate items and their counts:
Item: 1.28, Count: 4
Item: 19.123, Count: 4
Item: 75.93, Count: 4
Item: 19.130, Count: 4
Item: 19.124, Count: 4
Item: 19.134, Count: 4
Item: 1.139, Count: 4
Item: 19.114, Count: 4
Item: 75.52, Count: 4
Item: 1.130, Count: 4
Item: 19.131, Count: 4
Item: 1.30, Count: 4
Item: 19.120, Count: 2
Item: 19.115, Count: 3
Item: 34.184, Count: 4
Item: 19.144, Count: 3
Item: 19.188, Count: 4
Item: 8.22, Count: 4
Item: 1.125, Count: 4
Item: 19.129, Count: 2
Item: 34.176, Count: 3
Item: 34.82, Count: 2
Item: 8.24, Count: 2
Item: 34.115, Count: 2
Item: 1.119, Count: 2
Unique list: ['19.111', '1.139', '1.45', '34.35', '8.22', '34.115', '75.93', '19.116', '19.124', '19.144', '19.120', '19.131', '34.176', '1.121', '19.188', '75.9', '1.119', '34.82', '34.60', '75.38', '19.134', '19.114', '19.130', '1.11', '75.52', '19.156', '1.28', '8.24', '19.115', '8.27', '19.118', '75.167', '19.119', '34.184', '1.125', '19.129', '19.158', '1.130